In [1]:
# Imports
import cobra
from cobra.util.solver import linear_reaction_coefficients
import gurobipy
import cobra.util.array as cba
import pandas as pd
# STops the truncation of the results with "...""
pd.options.display.max_rows = 4000

import numpy as np

# for writing out files without overwriting other ones
from datetime import date
import os
import escher

# files for loopless and pFBA
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

# files for FVA
from cobra.flux_analysis import flux_variability_analysis

# for writing out excel files
import xlsxwriter

import datetime
from os import path

import math

In [2]:
def rxnBalanced(model):
    """if positive H:1 and charge:1 then you add a H+ to the reactant side, if H:-1 and charge: -1
    then you add the H+ to the product side"""

    l = len(model.reactions)
    counter = 0

    for x in range(l):
        rxn_id = model.reactions[x].id
        #  The dictionary with the mass balance
        mbD = model.reactions.get_by_id(rxn_id).check_mass_balance()

        if bool(mbD) == True:
            counter += 1

            print("The mass and/or charge is not balanced in rxn", rxn_id)
            print("Balance: ", mbD)
    print("Number of unbalanced reactions is: ", counter)

In [3]:
# function to print matrix as table

def Smatrix2table(model, matrix):
    '''this function returns a matrix written as a table with labels'''
    shape = matrix.shape
    nummetabolites = shape[0]
    numreactions = shape[1]
    listMet =[]
    listRxn = []
    for m in range(len(model.metabolites)):
        metid = model.metabolites[m].id
        listMet.append(metid)
    for r in range(len(model.reactions)):
        rxnid = model.reactions[r].id
        listRxn.append(rxnid)
    
    table = pd.DataFrame(matrix,index=listMet,columns=listRxn)
    return table

In [4]:
# function to print out outputs of optimization in a nice looking DF, returns an alphabetized table and a list to put in the original order

def solution_as_table(model, solution):
    
    #series
    OV = solution.objective_value
    SS = solution.status
    FX = solution.fluxes
    SP = solution.shadow_prices
    RC = solution.reduced_costs
    
    #dict
    FX_d = solution.fluxes.to_dict()
    SP_d = solution.shadow_prices.to_dict()
    RC_d = solution.reduced_costs.to_dict()
    
    #dataframe
    FX_df = pd.DataFrame.from_dict(FX_d,orient = "index")
    SP_df = pd.DataFrame.from_dict(SP_d,orient = "index")
    RC_df = pd.DataFrame.from_dict(RC_d,orient = "index")
    
    #initialize dataframe
    frames = [FX_df,RC_df,SP_df]
    df = pd.concat(frames,axis =1)#,keys=['Reactions','Reactions','Metabolites'])
    df.columns = ['Fluxes','Reduced Costs','Shadow Prices']
    
    #makes the order in a list format
    orderrxn = list(FX_d.keys())
    ordermet = list(SP_d.keys())
    order = orderrxn+ordermet
    
    #prints values outside of the dataframe
    print ("Objective Value:\t",OV)
    print ("Solution Status:\t",SS)
    return df, order

In [5]:
# function to print out outputs of optimization in a nice looking DF, returns an alphabetized table and a list to put in the original order

def solution_as_table_short(model, solution):
    
    #series
    OV = solution.objective_value
    SS = solution.status
    FX = solution.fluxes

    #dict
    FX_d = solution.fluxes.to_dict()
 
    #dataframe
    FX_df = pd.DataFrame.from_dict(FX_d,orient = "index")

    #initialize dataframe
    frames = [FX_df]#,RC_df,SP_df]
    df = pd.concat(frames,axis =1)#,keys=['Reactions','Reactions','Metabolites'])
    df.columns = ['Fluxes']#,'Reduced Costs','Shadow Prices']
    
    #makes the order in a list format
    orderrxn = list(FX_d.keys())
    order = orderrxn#+ordermet

    #prints values outside of the dataframe
    print ("Objective Value:\t",OV)
    print ("Solution Status:\t",SS)
    return df, order

In [6]:
def nonzero_flux(model,solution):
    #dict
    FX_d = solution.fluxes.to_dict()
    
    for key, value in FX_d.items() :
        if value != 0.0:
            rxn = model.reactions.get_by_id(key).name
            print ("\033[1m",key, ":","\033[0m",rxn, ":", value)

In [7]:
def makeMap(directory, mapname, model, solution):
    os.chdir(directory)
    print(os.getcwd())
    
    builder = escher.Builder(model=model)
    reactiondata = solution.fluxes
    index = reactiondata.index

    reactiondict = {}
    for i in range(len(reactiondata)):
        key = reactiondata.index[i]
        value = reactiondata.values[i]
        valueint = float(value)
        reactiondict[key] = valueint
    
        #set values to the new 
    print(reactiondict)
    b = escher.Builder(map_json=mapname,
                   reaction_data=reactiondict,
                   # change the default colors
                   #reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                    #               {'type': 'mean', 'color': '#0000dd', 'size': 20},
                     #              {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=False)
    return b   

In [46]:
# path to the model files
# path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_DH_model.xml"
# path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_ZZ_model.xml"
path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_MM_model.xml"

In [47]:
#  Reads in the model
model = cobra.io.read_sbml_model(path2model)
model

Name,iMMM22IC
Memory address,0x011338c29828
Number of metabolites,555
Number of reactions,619
Number of groups,104
Objective expression,1.0*EX_biomass_e - 1.0*EX_biomass_e_reverse_5e4c9
Compartments,"cytosol, extracellular"


In [48]:
print("Number of genes in the model: ",len(model.genes))

Number of genes in the model:  509


## Validate model

In [49]:
cobra.io.sbml.validate_sbml_model(path2model, check_model=True) # Use with the newer COBRA

(<Model iMMM22IC at 0x1133963d710>,
 {'SBML_FATAL': [],
  'SBML_ERROR': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': [],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

## Set solver

In [50]:
model.solver = "gurobi"
print("MODEL OBJECTIVE: \n",model.objective)
print("MODEL SOLVER: \n",model.solver)

MODEL OBJECTIVE: 
 Maximize
1.0*EX_biomass_e - 1.0*EX_biomass_e_reverse_5e4c9
MODEL SOLVER: 
 \ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  EX_biomass_e - EX_biomass_e_reverse_5e4c9
Subject To
 h2_c: - 2 MVHHDR + 2 MVHHDR_reverse_5da06 - FRH + FRH_reverse_2a109 - HMD
   + HMD_reverse_23491 + 2 CARS - 2 CARS_reverse_24ade + 2 GCARS
   - 2 GCARS_reverse_e0ab6 + 2 2GCARS - 2 2GCARS_reverse_c9167 + 2 PECARS
   - 2 PECARS_reverse_1b82c + 2 2PECARS - 2 2PECARS_reverse_18dce
   + 2 PICARS - 2 PICARS_reverse_1a275 + 2 2GPICARS
   - 2 2GPICARS_reverse_e6c45 + GPECARS - GPECARS_reverse_5eced + 2GPECARS
   - 2GPECARS_reverse_eeff2 + NIT1b2 - NIT1b2_reverse_497d5 - Eha
   + Eha_reverse_5bd35 - Ehb + Ehb_reverse_822cd + tH2 - tH2_reverse_484e3
   = 0
 fdxox_c: - MVHHDR + MVHHDR_reverse_5da06 + FWD - FWD_reverse_c9261 + CbiL
   - CbiL_reverse_f0d0a + 3 HTCAF - 3 HTCAF_reverse_9de33 + CoMF
   - CoMF_reverse_5047b + CODH2 - CODH2_reverse_44594 + CODHr2
   - 




## Checks For Rxn Balance

In [51]:
# Values lower than zero indicate missing atoms on the product side 
#whereas positive values indicate missing atoms on the substrate side
# to add a metabolite to the substrate side, the coefficient should be less than zero
# to add a metabolite to the product side, the coefficient should be greater than zero
rxnBalanced(model)

The mass and/or charge is not balanced in rxn PROTEIN_MM
Balance:  {'charge': -0.07839999999999492, 'C': -44.38330000000002, 'H': -88.98930000000001, 'N': -12.419299999999993, 'O': -22.22220000000013, 'S': -0.38570000000000004, 'P': -7.105427357601002e-15}
The mass and/or charge is not balanced in rxn DNA_MM
Balance:  {'charge': -0.0002999999999993008, 'C': -31.70360000000001, 'H': -39.827, 'N': -12.1633, 'O': -19.4957, 'P': -3.249200000000001}
The mass and/or charge is not balanced in rxn RNA_MM
Balance:  {'charge': -1.5543122344752192e-15, 'C': -29.621599999999994, 'H': -36.52440000000001, 'N': -11.8732, 'O': -21.78529999999999, 'P': -3.107199999999998}
The mass and/or charge is not balanced in rxn SMP_MM
Balance:  {'charge': -4.440899999999999, 'C': -26.574799999999996, 'H': -35.9709, 'N': -6.7433, 'Ni': -0.24509999999999998, 'O': -15.1221, 'P': -1.3828999999999998, 'S': -0.9665000000000001, 'Fe': -0.5281, 'Co': -0.489, 'Zn': -0.4247, 'Mg': -1.1429, 'Ca': -0.6931, 'K': -0.7105, 'Mo'

# Part 1: Prepare model for general use

# Turn off multiple option pathways (stylistic)

In [52]:
# reaction written together/apart (stylistic)
model.reactions.get_by_id("HISTDa").bounds = (0,0)
model.reactions.get_by_id("HISTDb").bounds = (0,0)
# model.reactions.get_by_id("HISTD").bounds = (0,0) # combined version

# model.reactions.CODH2.bounds = (0,0) # required to be on for growth on CO
# model.reactions.ACS3.bounds = (0,0) # required to be on for growth on CO
model.reactions.CODHr2.bounds = (0,0) # combined version (doesn't work for growth on CO)

# alternative to PFL, but accounting for activating enzymes (technically only present in DH)
model.reactions.get_by_id("PFL2").bounds = (0,0)


# Make model reaction wise microbe specific

In [53]:
# For ALL
# model.reactions.Ehb.bounds = (0,0) # want only one active (Eha)
# model.reactions.get_by_id("HMD").bounds = (-1000,1000) #(not sure if it should be zero) Also prevents loops
# model.reactions.get_by_id("MTD").bounds = (-1000,1000) # Also prevents loops
# model.reactions.FRH.bounds = (0,1000)
#--------------------------------------------------------------------------------------------------------------
# Make bounds (0,0) for DH and ZZ

# # Methanopepterin Biosynthesis (methylation at different point)
# # The following reactions that are turned off are for the Marburgensis, so when using MM we can comment them out
# model.reactions.get_by_id("DHPS3").bounds = (0,0) #(dhrfapF equivalent for Marburgensis)
# model.reactions.get_by_id("H4MPTS9").bounds = (0,0) # (dhadrpF equivalent for Marburgensis)
# model.reactions.get_by_id("dhadrF2").bounds = (0,0) # (dhadrF equivalent for Marburgensis)

# # Psuedomurein Biosynthesis (Galactosamine -Marburg vs glucosamine at different point)
# # The following reactions that are turned off are for the Z-245 and DH
# model.reactions.get_by_id("UGALNACS").bounds = (0,0) # UACNACS equivalent for Marburgensis
# model.reactions.get_by_id("PSMNS2").bounds = (0,0) # PSMNS equivalent for Marburgensis

#---------------------------------------------------------------------------------------------------------------
# Make bounds (0,0) for MM

# Methanopepterin Biosynthesis (methylation at different point)
# The following reactions that are turned off are for the Z-245 and DH, so when using DH/ZZ we can comment them out
model.reactions.get_by_id("dhrfapF").bounds = (0,0) #(DHPS3 equivalent for DH/Z-245)
model.reactions.get_by_id("dhadrpF").bounds = (0,0) # (H4MPTS9 equivalent for DH/Z-245)
model.reactions.get_by_id("dhadrF").bounds = (0,0) # (dhadrF2 equivalent for DH/Z-245)

# Psuedomurein Biosynthesis (Galactosamine -Marburg vs glucosamine at different point)
# The following reactions that are turned off are for the Marburgensis
model.reactions.get_by_id("UACNACS").bounds = (0,0) # UGALNACS equivalent for DH/Z-245
model.reactions.get_by_id("PSMNS").bounds = (0,0) # PSMNS2 equivalent for DH/Z-245

# homologus genes not found for MM
model.reactions.get_by_id("RNDR1").bounds = (0,0) # no gene in Marburg?
model.reactions.get_by_id("RNDR2").bounds = (0,0) # no gene in Marburg?
model.reactions.get_by_id("RNDR3").bounds = (0,0) # no gene in Marburg?
model.reactions.get_by_id("RNDR4").bounds = (0,0) # no gene in Marburg?
model.reactions.PFL.bounds = (0,0) # only in DH (0,1000) otherwise makes loop
model.reactions.PFL2.bounds = (0,0) # only in DH (0,1000) otherwise makes loop
#-----------------------------------------------------------------------------

# # Make bounds (0,0) for ZZ
# model.reactions.PFL.bounds = (0,0) # only in DH (0,1000) otherwise makes loop
# model.reactions.PFL2.bounds = (0,0) # only in DH (0,1000) otherwise makes loop

# Make bounds (0,0) for DH/MM and ZZ not grown on formate

# Energy Metabolism
model.reactions.get_by_id("FDH_F420").bounds = (0,0) # (-1000,1000) ONLY Z-245

# Define standard media with bounds differences by microbe
Doesn't include energy sources

In [54]:
# For all microbes make the bounds (0,0) unless added to media
model.reactions.get_by_id("EX_4abz_e").bounds = (0,0) # this can be used as alternative to 4hbz_c (BRAHPS) (if added to media)
model.reactions.get_by_id("EX_4hphac_e").bounds = (0,0) # assimiliation of 4-Hydroxyphenylacetate (if added to media, in maripaludis)
model.reactions.get_by_id("EX_btn_e").bounds = (0,0) # assimiliation biotin (if added to media)
model.reactions.get_by_id("EX_ind3ac_e").bounds = (0,0) # assimiliation of Indole-3-acetate (if added to media, in maripaludis)

# DH and ZZ can never use, but MM can when provided (not standard media)
model.reactions.get_by_id("EX_ppa_e").bounds = (0,0) # ONLY MARBURGENSIS - exchange of propanoate (if added to media)
model.reactions.get_by_id("EX_pyr_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of pyruvate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_5aop_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of 5 Aminolevulinate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_fum_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of fumarate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_succ_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of succinate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_pac_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of phenylacetate (if added to media, evidence only in Marburgensis)
model.reactions.get_by_id("EX_pro__L_e").bounds = (0,0) # none added to the model and none measured...
model.reactions.get_by_id("EX_ac_e").bounds = (0,0) # MARBURGENSIS and maybe Z-245 - assimiliation of acetate (if added to media, only in Marburgensis, and Z-245)

# Growth constrained Energy/Carbon/Sulfur sources

In [55]:
# Energy (electron) sources
model.reactions.get_by_id("EX_h2_e").bounds = (-1000,0)

# Carbon sources
model.reactions.get_by_id("EX_co2_e").bounds = (-1000,0)

# Energy + Carbon souces
#Growth on formate requires the FDH_F420 (-1000,1000) and for CO2 to be let out (0,1000)
model.reactions.get_by_id("EX_for_e").bounds = (0,0) # ONLY Z-245 (or Marburgensis for assimiliation/growth of formate (if added to media, evidence only in Marburgensis (assimilation) and Z-245 (growth))
model.reactions.get_by_id("EX_co_e").bounds = (0,0) # assimiliation carbon monoxide (if added to gas, only by Marburgensis)
# for growth on CO
# model.reactions.Eha.bounds = (0,1000) # (-1000,1000) for CO otherwise (0,1000)

# Sulfur Sources
model.reactions.get_by_id("EX_cys__L_e").bounds = (-0.0145,1000) # 0.0145 assimiliation of cysteine (H2S must be left open?)
model.reactions.get_by_id("EX_h2s_e").bounds = (-1000,1000) # 

# Nitrogen Sources
model.reactions.get_by_id("EX_nh3_e").bounds = (-1000,1000) # not quantifiable
model.reactions.get_by_id("EX_n2_e").bounds = (0,0) # assimiliation of nitrogen (if added to gas)

# Limit Products

In [56]:
model.reactions.get_by_id("EX_ch4_e").bounds = (0,1000)
model.reactions.get_by_id("EX_biomass_e").bounds = (0,1000)

# Initalize ATPM reaction

In [57]:
model.reactions.get_by_id("ATPM").bounds = (1.5,1000)

# End part 1 with general constraints
Using the constraints provided above, the model should be able to predict general phenotypes
It will consume lots of cysteine if it is not constrained.

# Part 2: Experimental Data

## FBA

In [58]:
# Define objective function
obj = {model.reactions.get_by_id("EX_biomass_e"):1}
# obj = {model.reactions.get_by_id("ATPM"):1}
model.objective = obj

In [59]:
# Defining ATPM
model.reactions.get_by_id("ATPM").bounds = (1.5,1000)
# model.reactions.get_by_id("ATPM").bounds = (23.5,23.5) #21.381

# ZZ
# model.reactions.get_by_id("ATPM").bounds = (15.01953,1000)

In [60]:
# From run F6_2
# Adjusted for dissolved CO2 but dropping all time points that are thought to be gross measurement error 
# AND the remaining points adjusted for mass balance (keeping dilution rate the same… not sure how to change that)

# DH AVG
# h2_exp = -175.41 #91.43
# co2_exp = -44.80 #22.87
# ch4_exp = 42.88 #22.85
# biomass_exp = 0.046 #0.002
# error = 0.001

# # ZZ AVG
# h2_exp = -117.73 #41.89
# #co2_exp = -28.47 #10.46 OLD old error 0.125
# #ch4_exp = 34.14 #6.05 OLD old error 0.125

# co2_exp = -30.37 #10.48
# ch4_exp = 28.47 #10.46
# biomass_exp = 0.045 #0.001
# error = 0.001

# MM AVG
h2_exp = -130.43 #17.12
co2_exp = -33.58 #4.28
ch4_exp = 31.61 #4.28
biomass_exp = 0.047 #0.001
error = 0.005

h2 = h2_exp
co2 = co2_exp
ch4 = ch4_exp
bm = biomass_exp


# Energy (electron) sources
model.reactions.get_by_id("EX_h2_e").bounds = (h2+h2*error,h2-h2*error)

# Carbon sources
model.reactions.get_by_id("EX_co2_e").bounds = (co2+co2*error,co2-co2*error)
model.reactions.get_by_id("EX_ac_e").bounds = (0,0)

# Energ + Carbon souces
#Formate requires the FDH_F420 (0,1000) and for CO2 to be let out
model.reactions.get_by_id("EX_for_e").bounds = (0,0)
model.reactions.get_by_id("EX_co_e").bounds = (0,0)

# Sulfur Sources
model.reactions.get_by_id("EX_cys__L_e").bounds = (-0.0145,1000)
model.reactions.get_by_id("EX_h2s_e").bounds = (-1000,1000)

# Nitrogen Sources
model.reactions.get_by_id("EX_nh3_e").bounds = (-1000,1000) # not quantifiable
model.reactions.get_by_id("EX_n2_e").bounds = (0,0) # assimiliation of nitrogen (if added to gas)

# Define products
model.reactions.get_by_id("EX_ch4_e").bounds = (ch4-ch4*error,ch4+ch4*error)
model.reactions.get_by_id("EX_biomass_e").bounds = (bm,bm)#(bm-bm*error,bm+bm*error)

In [61]:
solution = model.optimize()
print(solution)

<Solution 0.047 at 0x11339b9f3c8>


In [62]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
2dglc_e,EX_2dglc_e,6.354E-05,6,0.00%
ca2_e,EX_ca2_e,0.0002932,0,0.00%
co2_e,EX_co2_e,33.49,1,99.87%
cobalt2_e,EX_cobalt2_e,0.0002068,0,0.00%
cys__L_e,EX_cys__L_e,0.0145,3,0.13%
fe2_e,EX_fe2_e,0.0002234,0,0.00%
h2_e,EX_h2_e,130,0,0.00%
k_e,EX_k_e,0.0003005,0,0.00%
mg2_e,EX_mg2_e,0.0004834,0,0.00%
mobd_e,EX_mobd_e,8.705E-05,0,0.00%


In [63]:
solution = solution.to_frame()
solution

,fluxes,reduced_costs
MVHHDR,3.152981e+01,0.0
FRH,0.000000e+00,0.0
HMD,6.515792e+01,0.0
FTRM,3.217148e+01,0.0
FWD,3.217148e+01,0.0
MCH,3.217148e+01,0.0
MTD,-3.298644e+01,-0.0
MER,3.223226e+01,0.0
MTR,3.145195e+01,0.0
MCR,3.145195e+01,0.0


In [64]:
# Write out experimentally constrained model (matlab) to be used with in GIMME in the COBRA Toolbox
# cobra.io.save_matlab_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_DH_model_bound_adjval.mat")
# cobra.io.write_sbml_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_DH_model_bound_adjval.xml")
# cobra.io.save_json_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_DH_model_bound_adjval.json")

# cobra.io.save_matlab_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_ZZ_model_bound_adjval.mat")
# cobra.io.write_sbml_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_ZZ_model_bound_adjval.xml")
# cobra.io.save_json_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_ZZ_model_bound_adjval.json")

# cobra.io.save_matlab_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_MM_model_bound_adjval.mat")
# cobra.io.write_sbml_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_MM_model_bound_adjval.xml")
# cobra.io.save_json_model(model, r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA6\20220623_MM_model_bound_adjval.json")

## Display in Map

In [ ]:
path = r"F:\GEM_Models\Final_Model"
mapnormal = makeMap(path,"DH_map_final_final.json",model,solution)
# mapnormal = makeMap(path,"ZZ_map_final_final.json",model,solution)
# mapnormal = makeMap(path,"MM_map_final_final.json",model,solution)

mapnormal.display_in_browser()#notebook()#browser()

## Export to Text File

In [ ]:
with open(r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\FBA4\FBA_DH_Bio.txt", "w") as outfile:
    outfile.write(str(solution))
    outfile.close()

# Part 2.5 pFBA with Experimental Data

In [ ]:
# Define objective function
obj = {model.reactions.get_by_id("EX_biomass_e"):1}
# obj = {model.reactions.get_by_id("ATPM"):1}
model.objective = obj

In [ ]:
solutionpfba = pfba(model)
solutionpfba = solutionpfba.to_frame()
solutionpfba

## Display in Map

In [ ]:
path = r"F:\GEM_Models\Final_Model"
mapnormal = makeMap(path,"DH_map_final_final.json",model,solutionpfba)
# mapnormal = makeMap(path,"ZZ_map_final_final.json",model,solution)
# mapnormal = makeMap(path,"MM_map_final_final.json",model,solution)

mapnormal.display_in_browser()#notebook()#browser()

## Export to Text File

In [ ]:
# with open(r"C:\Users\Isabella\Downloads\pFBA_20211220.txt", "w") as outfile:
#     outfile.write(str(solutionpfba))
#     outfile.close()